In [2]:
class Data(object):
    
    """
    This class is resposible for data creation.
    ATTENTION: make sure that prettytable package is installed.
    
    Methods defined in the class:
    
    <data>.init() - is responsible for data initialization.

    Example:
    data = Data()
    data.init()
    
    <data>.show_all() - for data visualisation in table form.
    
    <data>.show_fin() - shows flows inputs.
    
    <data>.show_pf() - shows power flows.
    
    <data>.show_prout() - shows products outputs.
    
    <data>.show_facility() - shows table of inputed facilites.
    
    """
    
    field_names = [['N', 'Наименование', 'Величина'],
                   ['Наименование', 'Величина', 'Размерность'],
                   ['Название процесса', 'Номер процесса']
                  ]
    
    def __init__(self):
        from prettytable import PrettyTable
        import numpy as np
        
        self.facilities = dict()
        self.f_conditions = dict()
        
        self.t1 = PrettyTable()
        self.t1.title = "Таблица 1 - Исходные данные"
        self.t1.field_names = Data.field_names[0]
        
        self.t2 = PrettyTable()
        self.t2.title = "Таблица 2 - Процессы с вводом сырья"
        self.t2.field_names = Data.field_names[1]
        
        self.t3 = PrettyTable()
        self.t3.title = "Таблица 3 - Перетоки"
        self.t3.field_names = Data.field_names[1]
        
        self.t4 = PrettyTable()
        self.t4.title = "Таблица 4 - Отводы продукта"
        self.t4.field_names = Data.field_names[1]
        
        self.t5 = PrettyTable()
        self.t5.title = 'Таблица 5 - Введенные установки'
        self.t5.field_names = Data.field_names[2]
     
    def init(self):
        self.N = int(input('Введите количество процессов: '))
        for i in range(1, self.N+1):
            self.f_conditions.update({i: []})
        self.F, self.ZF = [0 for i in range(self.N)], [0 for i in range(self.N)]
        self.a = [[0 for i in range(self.N)] for i in range(self.N)]
        self.aw, self.zw = [[0 for i in range(10)] for i in range(self.N)], [[0 for i in range(10)] for i in range(self.N)]
        self.__inputs(group = 1)
        self.__inputs(group = 2)
        self.__inputs(group = 3)
            
        for i in range(self.N):
            facility = input('Введите название процесса №%s: ' %(i+1))
            self.facilities.update({i+1:facility})
            
        for key, value in self.facilities.items():
            self.t5.add_row([key, value])
    
    #private method for data 
    def __inputs(self, group):
        
        if group == 1:
            while True:
                process_number = int(input('Номер процесса с вводом сырья: '))
                if process_number == 0:
                    break
                self.F[process_number-1] = float(input('Количество сырья:'))
                self.ZF[process_number-1] = float(input('Цена 1 тонны сырья: '))
                
                self.t2.add_row(['Номер процесса с вводом сырья: ', process_number, ' '])
                self.t2.add_row(['Количество сырья: ', self.F[process_number-1], 'тонн'])
                self.t2.add_row(['Цена 1 тонны сырья: ', self.ZF[process_number-1], 'руб.'])
            print()
                
        elif group == 2:
            while True:
                process_number = int(input('Номер процесса из которого вых.част.поток: '))
                if process_number == 0:
                    break
            
                #Part-flow
                PF_input = int(input('Номер процесса в который вх.част.поток: '))
                self.a[process_number-1][PF_input-1] = float(input('Количество частичного потока: '))/100
                
                self.t3.add_row(['Номер процесса из которого вых.част.поток: ', process_number, ' '])
                self.t3.add_row(['Номер процесса в который вх.част.поток: ', PF_input, ' '])
                self.t3.add_row(['Количество частичного потока: ', round(self.a[process_number-1][PF_input-1]*100,3), '%'])
                self.f_conditions[process_number].append(PF_input)
            print()
        
        elif group == 3:
            while True:
                position = 0
                process_number = int(input('Номер процесса с отводом продукта: '))
                if process_number == 0:
                    break
                
                while True:
                    self.aw[process_number-1][position] = float(input('Количество %s-го частичного продукта: ' %(position+1) ))
                    if self.aw[process_number-1][position] == 0:
                        break
                    self.zw[process_number-1][position] = float(input('Цена 1 тонны частичного продукта: '))
                    
                    self.t4.add_row(['Количество %s-го частичного продукта: ', self.aw[process_number-1][position], '%'])
                    self.t4.add_row(['Цена 1 тонны частичного продукта: ', self.zw[process_number-1][position], 'руб'])
                    
                    position += 1
                
                self.t4.add_row(['Номер процесса с отводом продукта: ', process_number, ' '])
            print()
               
    def show_all(self):
        """Shows general Data table"""
        self.t1.add_row([1, 'Количество процессов', str(self.N)])
        print(self.t1)
        print(self.t2)
        print(self.t3)
        print(self.t4)
        print(self.t5)
        
    def show_fin(self):
        """This method shows table of flow inputs"""
        print(self.t2)
    
    def show_pf(self):
        """This method shows table of power inputs"""
        print(self.t3)
    
    def show_prout(self):
        """This method shows table of product's outputs"""
        print(self.t4)
    
    def show_facility(self):
        """Shows table of inputed facilities"""
        print(self.t5)

In [100]:
class Solver():
    
    """
    This class is responsible for calculation. Only Data object or nddaray type are accepted.
    -----------------------------------------------------------------------------------------
    ATTENTION: If input data nddaray type you must transfer 7 types of data to the class:
                
               1. data (n*n matrix) - part power flows (%)
               2. flow (vector) - feed stream (ton)
               3. flow_price (n*n matrix) - power flows prices (rub)
               4. product (n*n matrix) - part product flows (%)
               5. product_price (n*n matrix) - prices of product flows (rub)
               6. conditions (dict) - dependency between process and power flows
               7. facilities (dict) - names of processes
    ------------------------------------------------------------------------------------------
    Example of class initialization:
    
    Data Type:
    
        data = Data()
        data.init()
        model = Solver(data)
        
    ndarray Type:
    
        model = Solver(data, flow, flow_price, product, product_price, conditions, facilities)
    """
    
    __facilities = []
    __conditions = dict()
    __calculated = False
    __data = []
    __flow = []
    __flow_price = []
    __product = []
    __product_price = []
    
    def __init__(self, data, flow=None, flow_price=None,product=None, product_price=None, conditions=None, facilities=None):
        from prettytable import PrettyTable
        
        self.zwt = list()
        
        self.flow_results = {
            'Сырье (тонны)': [],
            'Сырье (%)': [],
            'Продукты (тонны)': [],
            'Продукты (%)': [],
            'Сырье в процессе (тонны)': [],
            'Сырье в процессе (%)': []
        }
        
        self.powers = []
        self.powers_percent = []
        
        self.flow_table = PrettyTable()
        self.flow_table.title = 'Результаты расчета по блокам'
        
        names = ['Номер процесса', 'Название процесса', 'F(т)', 'F(%)', 'W(т)', 'W(%)', 'Fp(т)', 'Fp(%)']
        self.flow_table.field_names = names
        
        self.power_table = PrettyTable()
        self.power_table.title = 'Результаты расчета по перетокам'
        self.power_table.field_names = ['№', 'Наименование', 'Тонны', '%']
        
        self.fpt = PrettyTable()
        self.fpt.title = 'Цены по блокам'
        self.fpt.field_names = ['№', 'Название процесса', 'F(руб)', 'F(тонна руб)', 'W(руб)', 'W(тонна руб)']
        
        self.ppt = PrettyTable()
        self.ppt.title = 'Цены по перетокам'
        self.ppt.field_names = ['№','Наименование','Тонны','%','Цена','Цена 1т']
        
        
        if self.__dtype(data) == 'ndarray':
            Solver.__data = data
            Solver.__flow = flow
            Solver.__flow_price = flow_price
            Solver.__facilities = facilities
            Solver.__conditions = conditions
            Solver.__product = product
            Solver.__product_price = product_price
    
    def __dtype(self, data):
        import numpy as np
        global A
        global E
        
        if type(data) == Data:
            Solver.__data = data.a
            Solver.__flow = data.F
            Solver.__flow_price = data.ZF
            Solver.__facilities = data.facilities
            Solver.__conditions = data.f_conditions
            Solver.__product = data.aw
            Solver.__product_price = data.zw
            
        elif type(data) == np.ndarray:
            return 'ndarray'
        
        else:
            raise ValueError('Only Data objects or numpy.ndarray are accepted')
            
            
        E = np.eye(len(Solver.__data), dtype=int)
        A = np.array(Solver.__data)
        A = A.T
    
    def __btable(self, *results, ptype = 'flow_power',):
        import numpy as np
        
        if ptype == 'flow_power':
            
            for i in range(len(self.flow_results['Продукты (тонны)'])):
                self.flow_table.add_row(
                    [i+1, 
                     Solver.__facilities[i+1], 
                     self.flow_results['Сырье (тонны)'][i], 
                     self.flow_results['Сырье в процессе (%)'][i],
                     self.flow_results['Продукты (тонны)'][i],
                     self.flow_results['Продукты (%)'][i],
                     self.flow_results['Сырье в процессе (тонны)'][i],
                     self.flow_results['Сырье в процессе (%)'][i]
                    ]
                )
            
            num = 0
            for key in Solver.__conditions.keys():
                for value in Solver.__conditions[key]:
                    self.power_table.add_row(
                        [num + 1,
                         'Переток из ' + str(key) + '-го процесса в ' + str(value) + '-й',
                         self.powers[num],
                         self.powers_percent[num]
                        ]
                    )
                    num+=1
                    
        if ptype == 'price':
            
            for i in range(len(self.flow_results['Продукты (тонны)'])):
                self.fpt.add_row([
                    i+1,
                    Solver.__facilities[i+1],
                    Solver.__flow[i],
                    Solver.__flow[i]*Solver.__flow_price[i],
                    self.zwt[i],
                    np.round(self.zwt[i]/self.flow_results['Продукты (тонны)'][i],2)
                ])
            
            num = 0
            for i in range(len(self.pecs)):
                for j in range(len(self.pecs[i])):
                    if self.pecs[i][j] != 0:
                        self.ppt.add_row([
                            num+1,
                            str(j+1)+'-й продукт '+str(i+1)+'-го процесса',
                            np.round(self.pecs[i][j], 2),
                            Solver.__product[i][j],
                            np.round(Solver.__product_price[i][j]*self.pecs[i][j],2),
                            Solver.__product_price[i][j]
                        ])
                        
                        num+=1
                        
    def calculate_flows(self):
        """Flow calculation"""
        
        import numpy as np
        
        flow_equation = -1*A+E
            
        try:
            self.result = np.linalg.solve(flow_equation, Solver.__flow)
        except np.linalg.LinAlgError:
            print('Singular Matrix. Please check matrix correctness.')
            
        for i in range(len(self.result)):
                
            self.flow_results['Сырье (тонны)'].append(Solver.__flow[i])
            self.flow_results['Сырье (%)'].append(
                round( (Solver.__flow[i]*100/sum(Solver.__flow)), 3)
            )
                
            self.flow_results['Сырье в процессе (тонны)'].append(round(self.result[i],3))
            self.flow_results['Сырье в процессе (%)'].append(
                round( (self.result[i]*100/sum(Solver.__flow)), 3)
            )
            
            Solver.__data = np.array(Solver.__data)
            self.flow_results['Продукты (тонны)'].append(
                np.round(self.result[i] * (1 - sum(Solver.__data[i])),2)
            )
                
            for k in range(len(self.result)):
                if Solver.__data[i][k] != 0:
                    self.powers.append(
                        np.round(self.result[i] * Solver.__data[i][k],2))

                    self.powers_percent.append(
                        np.round(Solver.__data[i][k]*100,3)
                    )
                
        for i in range(len(self.result)):
            self.flow_results['Продукты (%)'].append(
                np.round( (self.flow_results['Продукты (тонны)'][i]*100/sum(self.flow_results['Продукты (тонны)'])), 3)
            )
                            
        self.__btable(self.flow_results, self.powers, self.powers_percent, ptype = 'flow_power')
        
        Solver.__calculated = True
    
    def calculate_economics(self):
        """Responsible for econimic calculation"""
        import numpy as np
        assert Solver.__calculated == True, ('You must calculate flows prior to calculate economics')
        
        eco = (np.array(Solver.__product)*np.array(Solver.__product_price))
        
        for i in range(len(eco)):
            self.zwt.append(
                np.round(sum(eco[i])*np.array(self.flow_results['Продукты (тонны)'][i]/100),2)
            )
         
        self.pecs = (np.array(model.flow_results['Продукты (тонны)'])*np.array(data.aw).T).T/100
        
        self.__btable(ptype='price')
             
    def show_all(self):
        """Show all results in table form"""
        print(self.flow_table)
        print(self.power_table)
        print(self.fpt)
        print(self.ppt)
        
    def show_flows(self):
        """Shows flows calc in table form"""
        print(self.flow_table)
        print(self.power_table)
        
    def show_economics(self):
        """Shows economics calc in table form"""
        print(self.fpt)
        print(self.ppt)

In [92]:
class Samples(object):
    
    """
    Sample data.
    Usage example:
        s1 = Samples().first_data()
        part_flows = s1[0]
        feed = s1[1]
        feed_price = s1[2]
        product = s1[3]
        product_price = s1[4]
        conditions = s1[5]
        facilities = s1[6]
    """

    def first_data(self):
        import numpy as np
        data = np.array([[0.    , 0.125 , 0.064 , 0.096 , 0.081 , 0.    , 0.153 , 0.    ,
        0.174 , 0.    , 0.047 , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.414 , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.0016, 0.109 , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.189 , 0.    , 0.    ,
        0.077 , 0.077 , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.015 , 0.    , 0.    , 0.    , 0.414 ,
        0.    , 0.    , 0.    , 0.567 , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.018 , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.545 , 0.    , 0.    ,
        0.    , 0.437 , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.195 , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.913 , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.038 , 0.    , 0.    , 0.    , 0.16  , 0.634 ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.28  ,
        0.    , 0.    , 0.12  , 0.05  ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.167 , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.207 , 0.285 , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.207 , 0.452 , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.057 ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    ]])
        
        flow = [8000.0, 0, 0, 0, 0, 4000.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        flow_price = [50.0, 0, 0, 0, 0, 70.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        
        product = [[62.4, 37.6, 0.0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [82.4, 17.6, 0.0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [11.5, 21.1, 17.3, 50.1, 0.0, 0, 0, 0, 0, 0],
                     [4.0, 16.9, 14.2, 16.2, 48.7, 0.0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [10.8, 89.2, 0.0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [100.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0]]
        
        product_price = [[110.0, 114.0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [117.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [121.0, 160.0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [110.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [70.0, 125.0, 140.0, 160.0, 0, 0, 0, 0, 0, 0],
                         [60.0, 70.0, 125.0, 140.0, 160.0, 0, 0, 0, 0, 0],
                         [60.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [60.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [150.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [117.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [121.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [117.0, 110.0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [80.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [120.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [86.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [100.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [100.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [170.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [40.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
        
        conditions = {1: [2, 3, 4, 5, 7, 9, 11],
                         2: [],
                         3: [],
                         4: [],
                         5: [10],
                         6: [3, 4, 14, 17, 18],
                         7: [4, 8, 12],
                         8: [],
                         9: [4, 14, 18],
                         10: [],
                         11: [],
                         12: [11],
                         13: [15],
                         14: [11, 15, 16],
                         15: [16, 19, 20],
                         16: [],
                         17: [4, 13, 14],
                         18: [13, 14],
                         19: [20],
                         20: []}
        
        facilities = {1: 'C-100',
                     2: 'C-200',
                     3: 'C-300/2',
                     4: 'C-300/1',
                     5: 'C-400',
                     6: 'АВТ-4',
                     7: 'ВТ битум.',
                     8: 'УП битума',
                     9: 'ВТ',
                     10: 'Изомеризация',
                     11: 'Кат.риф.-2',
                     12: 'Ги.крекинг',
                     13: 'Гид.очис.2',
                     14: 'Гид.очис.1',
                     15: 'Кат.крек',
                     16: 'Техн.топл.',
                     17: 'УЗК-1',
                     18: 'УЗК-2',
                     19: 'Алкилирование',
                     20: 'Утил.гудр'}
        
        return (data, flow, flow_price, product, product_price, conditions, facilities)
        

In [6]:
data = Data()

In [7]:
data.init()

Введите количество процессов: 20
Номер процесса с вводом сырья: 1
Количество сырья:8000
Цена 1 тонны сырья: 50
Номер процесса с вводом сырья: 6
Количество сырья:4000
Цена 1 тонны сырья: 70
Номер процесса с вводом сырья: 0

Номер процесса из которого вых.част.поток: 1
Номер процесса в который вх.част.поток: 2
Количество частичного потока: 12.5
Номер процесса из которого вых.част.поток: 1
Номер процесса в который вх.част.поток: 3
Количество частичного потока: 6.4
Номер процесса из которого вых.част.поток: 1
Номер процесса в который вх.част.поток: 4
Количество частичного потока: 9.6
Номер процесса из которого вых.част.поток: 1
Номер процесса в который вх.част.поток: 5
Количество частичного потока: 8.1
Номер процесса из которого вых.част.поток: 1
Номер процесса в который вх.част.поток: 7
Количество частичного потока: 15.3
Номер процесса из которого вых.част.поток: 1
Номер процесса в который вх.част.поток: 9
Количество частичного потока: 17.4
Номер процесса из которого вых.част.поток: 1
Ном

Введите название процесса №4: C-300/1
Введите название процесса №5: C-400
Введите название процесса №6: АВТ-4
Введите название процесса №7: ВТ битум.
Введите название процесса №8: УП битума
Введите название процесса №9: ВТ
Введите название процесса №10: Изомеризация
Введите название процесса №11: Кат.риф.-2
Введите название процесса №12: Ги.крекинг
Введите название процесса №13: Гид.очис.2
Введите название процесса №14: Гид.очис.1
Введите название процесса №15: Кат.крек
Введите название процесса №16: Техн.топл.
Введите название процесса №17: УЗК-1
Введите название процесса №18: УЗК-2
Введите название процесса №19: Алкилирование
Введите название процесса №20: Утил.гудр


In [8]:
data.show_all()

+-------------------------------------+
|     Таблица 1 - Исходные данные     |
+---+----------------------+----------+
| N |     Наименование     | Величина |
+---+----------------------+----------+
| 1 | Количество процессов |    20    |
+---+----------------------+----------+
+----------------------------------------------------------+
|           Таблица 2 - Процессы с вводом сырья            |
+---------------------------------+----------+-------------+
|           Наименование          | Величина | Размерность |
+---------------------------------+----------+-------------+
| Номер процесса с вводом сырья:  |    1     |             |
|        Количество сырья:        |  8000.0  |     тонн    |
|       Цена 1 тонны сырья:       |   50.0   |     руб.    |
| Номер процесса с вводом сырья:  |    6     |             |
|        Количество сырья:        |  4000.0  |     тонн    |
|       Цена 1 тонны сырья:       |   70.0   |     руб.    |
+---------------------------------+----------+---

In [71]:
model = Solver(data)

In [72]:
model.calculate_flows()

In [73]:
model.calculate_economics()

C:\Users\Khliv\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:120: RuntimeWarning: invalid value encountered in double_scalars


In [74]:
model.show_all()

+---------------------------------------------------------------------------------------------+
|                                 Результаты расчета по блокам                                |
+----------------+-------------------+--------+--------+---------+--------+----------+--------+
| Номер процесса | Название процесса |  F(т)  |  F(%)  |   W(т)  |  W(%)  |  Fp(т)   | Fp(%)  |
+----------------+-------------------+--------+--------+---------+--------+----------+--------+
|       1        |       C-100       | 8000.0 | 66.667 |  2080.0 | 17.333 |  8000.0  | 66.667 |
|       2        |       C-200       |   0    | 8.333  |  1000.0 | 8.333  |  1000.0  | 8.333  |
|       3        |      C-300/2      |   0    |  4.32  |  518.4  |  4.32  |  518.4   |  4.32  |
|       4        |      C-300/1      |   0    | 10.824 | 1298.85 | 10.824 | 1298.852 | 10.824 |
|       5        |       C-400       |   0    |  5.4   |  379.73 | 3.164  |  648.0   |  5.4   |
|       6        |       АВТ-4       | 4

In [93]:
s1 = Samples().first_data()
part_flows = s1[0]
feed = s1[1]
feed_price = s1[2]
product = s1[3]
product_price = s1[4]
conditions = s1[5]
facilities = s1[6]

In [101]:
model2 = Solver(
    data=part_flows, flow=feed, flow_price=feed_price, product=product, product_price=product_price,
    conditions=conditions, facilities=facilities)

In [103]:
model2.calculate_flows()

In [105]:
model2.calculate_economics()

C:\Users\Khliv\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:147: RuntimeWarning: invalid value encountered in double_scalars


In [106]:
model2.show_flows()

+---------------------------------------------------------------------------------------------+
|                                 Результаты расчета по блокам                                |
+----------------+-------------------+--------+--------+---------+--------+----------+--------+
| Номер процесса | Название процесса |  F(т)  |  F(%)  |   W(т)  |  W(%)  |  Fp(т)   | Fp(%)  |
+----------------+-------------------+--------+--------+---------+--------+----------+--------+
|       1        |       C-100       | 8000.0 | 66.667 |  2080.0 | 17.333 |  8000.0  | 66.667 |
|       2        |       C-200       |   0    | 8.333  |  1000.0 | 8.333  |  1000.0  | 8.333  |
|       3        |      C-300/2      |   0    |  4.32  |  518.4  |  4.32  |  518.4   |  4.32  |
|       4        |      C-300/1      |   0    | 10.824 | 1298.85 | 10.824 | 1298.852 | 10.824 |
|       5        |       C-400       |   0    |  5.4   |  379.73 | 3.164  |  648.0   |  5.4   |
|       6        |       АВТ-4       | 4

In [107]:
model2.show_economics()

+---------------------------------------------------------------------------+
|                               Цены по блокам                              |
+----+-------------------+--------+--------------+-----------+--------------+
| №  | Название процесса | F(руб) | F(тонна руб) |   W(руб)  | W(тонна руб) |
+----+-------------------+--------+--------------+-----------+--------------+
| 1  |       C-100       | 8000.0 |   400000.0   | 231928.32 |    111.5     |
| 2  |       C-200       |   0    |      0       |  117000.0 |    117.0     |
| 3  |      C-300/2      |   0    |      0       |  66284.7  |    127.86    |
| 4  |      C-300/1      |   0    |      0       |  142873.5 |    110.0     |
| 5  |       C-400       |   0    |      0       |  52708.42 |    138.8     |
| 6  |       АВТ-4       | 4000.0 |   280000.0   | 289766.85 |    132.58    |
| 7  |     ВТ битум.     |   0    |      0       |   294.0   |     60.0     |
| 8  |     УП битума     |   0    |      0       |  30404.4  |  